# Free Variable and Closure



In [1]:
def outer():
    x ="python"
    def inner():
        print(f"{x} rocks!")
    inner()

1. When the outer is defined , python see that x is shared between the two scope.
2. so , it will create the *intermediate cell object* two scope will point to this cell.
3. This cell have the memory address of the original "python" address.

In [2]:
outer.__code__.co_cellvars

('x',)

**Why we need this?**
1. consider we are return the inner function from the outer function.
2. after the outer function is executed outer will become the *out of scope.*
3. but inner need the outer x to print.
4. to avoid the x to become the out of scope . python will create the intermediate cell outer.x and inner.x will point the intermediate cell , then the intermediate cell will point to the original object.
5. inner.x is called free variable
6. since outer.x need for the inner() to execute the , *free variable is enclosed with inner function*.
This enclosing concept is called the closure.


In [3]:
def outer1():
    x = "python"
    def inner():
        print(f"{x} rocks")
    return inner

In [4]:
fn = outer1()
#! outer1 finished running , now outer1 will become the out of scope
fn()
#? then how does the "python" is available to fn
#* This is because of the closure. [inner() function + enclosing free variable]. this return from the outer function

python rocks


<img src="image/img.png">

# Closure

1. Function plus and extended scope that contain the free variable

In [5]:
def outer2():
    a =10
    x = "python"
    def inner():
        a =10
        print(f"{x} rocks")
    return inner

In [6]:
outer2.__code__.co_varnames

('a', 'inner')

In [7]:
outer2.__code__.co_cellvars
#? we can see that "a" is not part of the cellvars , since we assigned the inside the inner function

('x',)

In [8]:
fn = outer2()

In [9]:
fn.__code__.co_freevars
#! we can see that "x" in free var , because the outer2 become the out of scope.
#! the closure capture the free variable and inner function

('x',)

In [10]:
fn.__closure__
#? we can see that intermediate cell pointing to original object we define

(<cell at 0x00000203EF1D4EE0: str object at 0x00000203EC155C30>,)

In [11]:
def outer2():
    a =10
    x = "python"
    print(hex(id(x)))
    def inner():
        a =10
        print(f"{x} rocks")
        print(hex(id(x)))
    return inner

In [12]:
fn = outer2()

0x203ec155c30


In [13]:
fn.__closure__
#? in the outer function the "x" is pointing to the cell var , when we print the id of the x , it prints original address .
#? python does all the heavy-lifting for us.
#* when we try to access the original object python "double hop" cell -> original
#* python hides there exist the intermediate cell

(<cell at 0x00000203EF1D6980: str object at 0x00000203EC155C30>,)

# Modifying the free variables


In [14]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count +=1
        return count
    return inc

In [15]:
fn = counter()

In [16]:
fn.__closure__

(<cell at 0x00000203EF1D7400: int object at 0x00007FFD2117D308>,)

In [17]:
fn.__code__.co_freevars

('count',)

In [18]:
fn()

1

In [19]:
fn()

2

# Multiple instance of closure

In [20]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [21]:
#? first call will its own scope is created
fn1 = counter()
#? second call will have its own scope is created when the function is called
fn2 = counter()

In [22]:
#? here we have two instances of closure
print(fn2.__closure__)
print(fn1.__closure__)
#? since int 0 its is singleton object that why we have same memory address for int
#! closure have different memory address because whenever the function is called it will create the new scope

(<cell at 0x00000203EF1D7AF0: int object at 0x00007FFD2117D308>,)
(<cell at 0x00000203EF1D78E0: int object at 0x00007FFD2117D308>,)


In [23]:
print(fn1())
print(fn1())

1
2


In [24]:
print(fn2())
print(fn2())

1
2


# Shared Extended Scope

In [25]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count +=1
        return count
    def inc2():
        nonlocal count
        count += 1
        return count
    return inc1,inc2

In [26]:
fn1 ,fn2 = outer()

In [27]:
print(fn1.__closure__)
print(fn2.__closure__)
#!  both fn1,fn2 are pointing to same cell.

(<cell at 0x00000203EF228F70: int object at 0x00007FFD2117D308>,)
(<cell at 0x00000203EF228F70: int object at 0x00007FFD2117D308>,)


In [28]:
print(fn1())
print(fn1())

1
2


In [29]:
print(fn2())
print(fn2())

3
4


# Closure Application1

We can create closure in place of class

In [30]:
#? We want to calculate the average when we add the number
class Averager:
    def __init__(self):
        self.numbers =[]
    def add(self,number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total/count

In [31]:
a = Averager()

In [32]:
a.add(10) , a.add(20) ,a.add(30)

(10.0, 15.0, 20.0)

In [33]:
# we have same function using hte closure
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total/count
    return add

In [34]:
a = averager()

In [35]:
a.__closure__

(<cell at 0x00000203EF228DF0: list object at 0x00000203EDCBC2C0>,)

In [36]:
a(10),a(20),a(30)

(10.0, 15.0, 20.0)

In [37]:
#? since sum again and again
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total/count
    return add

In [38]:
a = averager()

In [39]:
a.__closure__,a.__code__.co_freevars

((<cell at 0x00000203EF22ADD0: int object at 0x00007FFD2117D308>,
  <cell at 0x00000203EF229FC0: int object at 0x00007FFD2117D308>),
 ('count', 'total'))

In [40]:
a(10),a(20),a(30)

(10.0, 15.0, 20.0)

In [41]:
#? we can also create the timer
from time import perf_counter
class Timer:
    def __init__(self):
        self.start = perf_counter()
    def __call__(self, *args, **kwargs):
        return perf_counter() - self.start

In [42]:
t1 = Timer()

In [43]:
t1()

0.010287600045558065

In [44]:
t1()

0.025447599997278303

In [45]:
t1()

0.039147500006947666

In [46]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() -start
    return poll

In [47]:
t2 = timer()

In [48]:
t2()

0.011886299995239824

In [49]:
t2()

0.02499940001871437

In [50]:
t2()

0.039445400005206466

# Counter using Closure

In [51]:
def counter(initial_value=0):
    def step(increment=1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return step


In [52]:
counter1 = counter()

In [53]:
counter1(),counter1(),counter1(),counter1()

(1, 2, 3, 4)

In [54]:
#? we also count number of time function os called
def counter(fn):
    cnt = 0
    def inner(*args,**kwargs):
        nonlocal cnt
        cnt += 1
        print(f"{fn.__name__} is called {cnt} times")
        return fn(*args,**kwargs)
    return inner



In [55]:
def add(a,b):
    return a+b

In [56]:
def mul(a,b):
    return a*b

In [57]:
counter_add = counter(add)
counter_mul = counter(mul)

In [58]:
counter_add(1,2)
counter_mul(1,2)

add is called 1 times
mul is called 1 times


2

In [59]:
counter_add(1,2)
counter_mul(1,2)

add is called 2 times
mul is called 2 times


2

In [60]:
counter_add(1,2)
counter_mul(1,2)

add is called 3 times
mul is called 3 times


2

In [61]:
func_counter = dict()

In [62]:
#? instead of printing the count we need to store them in dict
def counter(fn):
    cnt  =0
    def inner(*arg,**kwargs):
        nonlocal cnt
        cnt += 1
        func_counter[fn.__name__] = cnt
        return fn(*arg,**kwargs)
    return inner



In [63]:
counter_mul = counter(mul)
counter_add = counter(add)


In [64]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [65]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [66]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [67]:
func_counter

{'add': 3, 'mul': 3}

In [68]:
def counter(fn,func_counter):
    cnt  =0
    def inner(*arg,**kwargs):
        nonlocal cnt
        cnt += 1
        func_counter[fn.__name__] = cnt
        return fn(*arg,**kwargs)
    return inner


In [69]:
fn_counter = {}
counter_add = counter(add,fn_counter)
counter_mul = counter(mul,fn_counter)

In [70]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [71]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [72]:
counter_add(1,2),counter_mul(1,2)

(3, 2)

In [73]:
fn_counter

{'add': 3, 'mul': 3}

In [74]:
fn_counter = {}
add = counter(add,fn_counter)
mul = counter(mul,fn_counter)

In [75]:
add(1,2),mul(1,2)

(3, 2)

In [76]:
add(1,2),mul(1,2)

(3, 2)

In [77]:
add(1,2),mul(1,2)

(3, 2)

In [78]:
add(1,2),mul(1,2)

(3, 2)

In [79]:
fn_counter

{'add': 4, 'mul': 4}